In [12]:
import os
import json
import asyncio
from pprint import pprint
from datetime import datetime, timedelta
import nest_asyncio
nest_asyncio.apply()

> DEFAULT_AUTH_TOKEN

In [ ]:
# import base64

# username = "pooa8ec-evyh6w8bb-d8h6t6l-1hyxf0a0gt"
# password = "K@flgj61d0V^Bm+#b!aO&jSOVM$Fch"
# credentials = f"{username}:{password}"

# token = base64.b64encode(credentials.encode()).decode()
# print(token)

cG9vYThlYy1ldnloNnc4YmItZDhoNnQ2bC0xaHl4ZjBhMGd0OktAZmxnajYxZDBWXkJtKyNiIWFPJmpTT1ZNJEZjaA==


In [20]:
import base64
import requests # We need this library to make a web request
from pprint import pprint
from ratelimit import limits, sleep_and_retry

# Define your desired rate limit: 12 calls per second
CALLS = 12
PERIOD = 1

def create_auth_headers(username, password):
    """Encodes credentials into a Basic Auth header."""
    auth_string = f'{username}:{password}'.encode('utf-8')
    auth_encoded = base64.b64encode(auth_string).decode('utf-8')
    return {'Authorization': f'Basic {auth_encoded}'}


# Apply the rate limit decorator to the function that performs the action
@sleep_and_retry
@limits(calls=CALLS, period=PERIOD)
def make_api_request(url, headers):
    """
    Makes a rate-limited API request and returns the response.
    This function will automatically sleep if it's called too frequently.
    """
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status() # Raise an exception for bad status codes (4xx or 5xx)
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# --- Example Usage ---
my_username = "pooa8ec-evyh6w8bb-d8h6t6l-1hyxf0a0gt"
my_password = "K@flgj61d0V^Bm+#b!aO&jSOVM$Fch"
some_api_url = "https://apirest.wyscout.com/v2" # Replace with a real API endpoint

# 1. Create the headers
auth_headers = create_auth_headers(my_username, my_password)
print("Generated Headers:")
pprint(auth_headers)
# 2. Call the rate-limited function (e.g., in a loop)
print(f"Making requests to {some_api_url} (limited to {CALLS} per second)...")
for i in range(5):
    print(f"Request #{i+1}...")
    data = make_api_request(some_api_url, auth_headers)
    if data:
        pprint(data)

Generated Headers:
{'Authorization': 'Basic '
                  'cG9vYThlYy1ldnloNnc4YmItZDhoNnQ2bC0xaHl4ZjBhMGd0OktAZmxnajYxZDBWXkJtKyNiIWFPJmpTT1ZNJEZjaA=='}
Making requests to https://apirest.wyscout.com/v2 (limited to 12 per second)...
Request #1...
An error occurred: 403 Client Error: Forbidden for url: https://apirest.wyscout.com/v2
Request #2...
An error occurred: 403 Client Error: Forbidden for url: https://apirest.wyscout.com/v2
Request #3...
An error occurred: 403 Client Error: Forbidden for url: https://apirest.wyscout.com/v2
Request #4...
An error occurred: 403 Client Error: Forbidden for url: https://apirest.wyscout.com/v2
Request #5...
An error occurred: 403 Client Error: Forbidden for url: https://apirest.wyscout.com/v2


In [21]:
from backend.agents.wyscout.tools import (wyscout_advanced_stats, wyscout_area_list, wyscout_coach_info, 
wyscout_competition_info, wyscout_match_events, wyscout_match_info, wyscout_player_info, wyscout_referee_info,  wyscout_round_info,
wyscout_id_search, wyscout_season_info, wyscout_team_info, wyscout_video_tool )

In [5]:
import os
import json
import asyncio
from pprint import pprint
from datetime import datetime, timedelta
import nest_asyncio
nest_asyncio.apply()

# This script assumes that you have a file structure like:
# /your_project_folder
# |--- /backend
# |    |--- /agents
# |         |--- /wyscout
# |              |--- tools.py  <-- All your tool definitions are in here
# |--- test_all_tools.py      <-- This is the current script

# To run this script, your tool definitions must be in the correct path.
# If your structure is different, you may need to adjust the import statement.
try:
    from backend.agents.wyscout.tools import (
        wyscout_id_search, wyscout_player_info, wyscout_team_info,
        wyscout_competition_info, wyscout_season_info, wyscout_match_info,
        wyscout_advanced_stats, wyscout_match_events, wyscout_video_tool,
        wyscout_area_list, wyscout_round_info, wyscout_coach_info,
        wyscout_referee_info
    )
except ImportError:
    print("🚨 ImportError: Could not import the Wyscout tools.")
    print("Please ensure this script is run from the root of your project and that the tool definitions")
    print("are located at: backend/agents/wyscout/tools.py")
    exit()


def print_header(title):
    """Helper function to print a formatted header for each test section."""
    print("\n" + "="*80)
    print(f"  🧪 TESTING: {title}")
    print("="*80)

async def main():
    """
    Main asynchronous function to orchestrate the testing of all Wyscout tools.
    """
    print("🚀 Starting Wyscout Tool Test Suite...")
    
    # --- [1] Dynamic ID Fetching ---
    # We must run search first to get IDs for other tests.
    print_header("wyscout_id_search")
    player_id, team_id, competition_id, season_id, match_id = None, None, None, None, None
    
    try:
        print("\n--- Searching for Player: Lionel Messi ---")
        result = await wyscout_id_search.arun({"search_term": "Lionel Messi", "entity_type": "player", "limit": 1, "gender": None})
        player_id = result.get("potential_matches", [{}])[0].get("wyId")
        print(f"✅ Found Player ID: {player_id}")
        pprint(result)

        print("\n--- Searching for Team: Real Madrid ---")
        result = await wyscout_id_search.arun({"search_term": "Real Madrid", "entity_type": "team", "limit": 1, "gender": None})
        team_id = result.get("potential_matches", [{}])[0].get("wyId")
        print(f"✅ Found Team ID: {team_id}")
        pprint(result)

        print("\n--- Searching for Competition: Premier League ---")
        result = await wyscout_id_search.arun({"search_term": "Premier League", "entity_type": "competition", "limit": 1, "gender": None})
        competition_id = result.get("potential_matches", [{}])[0].get("wyId")
        print(f"✅ Found Competition ID: {competition_id}")
        pprint(result)
        
    except Exception as e:
        print(f"🚨 CRITICAL ERROR during ID Search: {e}. Cannot proceed with most tests.")
        return

    # --- [2] Individual Tool Tests ---
    
    # Test Area Tool
    try:
        print_header("wyscout_area_list")
        result = await wyscout_area_list.arun({})
        print("✅ Successfully called. Result snippet:")
        # Print a small part of the large response
        pprint({k: v[:2] if isinstance(v, list) else v for k, v in result.items()})
    except Exception as e:
        print(f"🚨 FAILED: {e}")

    # Test Player Tool
    if player_id:
        try:
            print_header(f"wyscout_player_info (ID: {player_id})")
            result = await wyscout_player_info.arun({
                "wyId": player_id,
                "get_details": True,
                "get_career": True
            })
            print("✅ Successfully called.")
            pprint(result)
        except Exception as e:
            print(f"🚨 FAILED: {e}")

    # Test Team Tool
    if team_id:
        try:
            print_header(f"wyscout_team_info (ID: {team_id})")
            result = await wyscout_team_info.arun({
                "wyId": team_id,
                "get_squad": True,
                "get_fixtures": True,
                "from_date": (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d'),
                "to_date": (datetime.now() + timedelta(days=30)).strftime('%Y-%m-%d'),
            })
            print("✅ Successfully called.")
            pprint(result)
        except Exception as e:
            print(f"🚨 FAILED: {e}")
            
    # Test Competition Tool to get a Season ID
    if competition_id:
        try:
            print_header(f"wyscout_competition_info (ID: {competition_id})")
            result = await wyscout_competition_info.arun({
                "wyId": competition_id,
                "get_seasons": True,
                "active_seasons_only": True
            })
            season_id = result.get('seasons', {}).get('seasons', [{}])[0].get('wyId')
            print(f"✅ Found active Season ID: {season_id}")
            pprint(result)
        except Exception as e:
            print(f"🚨 FAILED: {e}")
            
    # Test Season Tool to get a Match ID
    if season_id:
        try:
            print_header(f"wyscout_season_info (ID: {season_id})")
            result = await wyscout_season_info.arun({
                "wyId": season_id,
                "get_matches": True
            })
            match_id = result.get('matches', {}).get('matches', [{}])[0].get('matchId')
            print(f"✅ Found Match ID: {match_id}")
            pprint(result)
        except Exception as e:
            print(f"🚨 FAILED: {e}")
            
    # Test Match, Events, Advanced Stats, and Video tools with the Match ID
    if match_id:
        try:
            print_header(f"wyscout_match_info (ID: {match_id})")
            result = await wyscout_match_info.arun({
                "wyId": match_id,
                "get_details": True,
                "details_relations": ["competition", "teams"]
            })
            print("✅ Successfully called.")
            pprint(result)
        except Exception as e:
            print(f"🚨 FAILED: {e}")

        try:
            print_header(f"wyscout_match_events (ID: {match_id})")
            result = await wyscout_match_events.arun({
                "match_id": match_id,
                "filter_by_primary_types": ["shot"],
                "filter_by_period": ["1H"]
            })
            print("✅ Successfully called (filtered for 1H shots).")
            pprint(result)
        except Exception as e:
            print(f"🚨 FAILED: {e}")
            
        try:
            print_header(f"wyscout_advanced_stats (Match Context - ID: {match_id})")
            result = await wyscout_advanced_stats.arun({
                "match_context": {
                    "match_id": match_id,
                    "get_team_level_stats": True
                }
            })
            print("✅ Successfully called.")
            pprint(result)
        except Exception as e:
            print(f"🚨 FAILED: {e}")

        try:
            print_header(f"wyscout_video_tool (SAFE calls - ID: {match_id})")
            result = await wyscout_video_tool.arun({
                "match_id": match_id,
                "check_available_qualities": True,
                "check_period_offsets": True
            })
            print("✅ Successfully called for safe info.")
            pprint(result)
            print("\n--- NOTE: Did not test costly 'generate_video_links' action. ---")
        except Exception as e:
            print(f"🚨 FAILED: {e}")
            
    print("\n✅ Test Suite Finished.")

# To run this script in a Jupyter Notebook, place the following
# command in a cell and execute it:
# await main()
#
# To run this as a standard Python file, use the following:
if __name__ == "__main__":
    asyncio.run(main())


🚀 Starting Wyscout Tool Test Suite...

  🧪 TESTING: wyscout_id_search

--- Searching for Player: Lionel Messi ---
✅ Found Player ID: None
{'error': 'API Error: 403', 'message': 'Forbidden'}

--- Searching for Team: Real Madrid ---
✅ Found Team ID: None
{'error': 'API Error: 403', 'message': 'Forbidden'}

--- Searching for Competition: Premier League ---
✅ Found Competition ID: None
{'error': 'API Error: 400', 'message': 'Bad Request'}

  🧪 TESTING: wyscout_area_list
✅ Successfully called. Result snippet:
{'documented_custom_areas': [{'alpha2code': 'ASX',
                              'alpha3code': 'AS',
                              'id': 1,
                              'name': 'Asia'},
                             {'alpha2code': 'AFX',
                              'alpha3code': 'AF',
                              'id': 2,
                              'name': 'Africa'}],
 'live_api_areas': {'areas': [{'alpha2code': 'AS',
                               'alpha3code': 'XAS',
          

> 1. wyscout_id_search

In [22]:
import asyncio
from pprint import pprint

# Ensure your tools are importable from your notebook's location
try:
    from backend.agents.wyscout.tools import wyscout_id_search
except ImportError as e:
    print(f"🚨 ImportError: {e}. Make sure your notebook is running from the project root.")

def print_header(title):
    print("\n" + "="*80)
    print(f"  🧪 TESTING: {title}")
    print("="*80)

async def test_id_search():
    print_header("wyscout_id_search")
    
    # --- Search for a Player ---
    print("\n--- Searching for Player: Cristiano Ronaldo ---")
    player_result = await wyscout_id_search.arun({
        "search_term": "Cristiano Ronaldo", "entity_type": "player", 
        "limit": 1, "gender": "men"
    })
    player_id = player_result.get("potential_matches", [{}])[0].get("wyId")
    print(f"✅ Found Player ID: {player_id}")
    pprint(player_result)

    # --- Search for a Team ---
    print("\n--- Searching for Team: FC Barcelona ---")
    team_result = await wyscout_id_search.arun({
        "search_term": "Barcelona", "entity_type": "team", 
        "limit": 1, "gender": None
    })
    team_id = team_result.get("potential_matches", [{}])[0].get("wyId")
    print(f"✅ Found Team ID: {team_id}")
    pprint(team_result)

    # --- Search for a Competition ---
    print("\n--- Searching for Competition: La Liga ---")
    competition_result = await wyscout_id_search.arun({
        "search_term": "La Liga", "entity_type": "competition", 
        "limit": 1, "gender": None
    })
    competition_id = competition_result.get("potential_matches", [{}])[0].get("wyId")
    print(f"✅ Found Competition ID: {competition_id}")
    pprint(competition_result)
    
    return player_id, team_id, competition_id

# Run the test and store the IDs for the next cells
# In Jupyter, 'await' can be used at the top level in a cell
player_id, team_id, competition_id = await test_id_search()



  🧪 TESTING: wyscout_id_search

--- Searching for Player: Cristiano Ronaldo ---
✅ Found Player ID: None
{'error': 'API Error: 403', 'message': 'Forbidden'}

--- Searching for Team: FC Barcelona ---
✅ Found Team ID: None
{'error': 'API Error: 403', 'message': 'Forbidden'}

--- Searching for Competition: La Liga ---
✅ Found Competition ID: None
{'error': 'API Error: 400', 'message': 'Bad Request'}


> 2. wyscout_player_info

In [23]:
import asyncio
from pprint import pprint

try:
    from backend.agents.wyscout.tools import wyscout_player_info
except ImportError as e:
    print(f"🚨 ImportError: {e}")

def print_header(title):
    print("\n" + "="*80)
    print(f"  🧪 TESTING: {title}")
    print("="*80)

async def test_player_tool(p_id):
    if not p_id:
        print("\nSkipping Player Tool test: No Player ID found from the previous cell.")
        return

    print_header(f"wyscout_player_info (ID: {p_id})")
    
    result = await wyscout_player_info.arun({
        "wyId": p_id,
        "get_details": True,
        "get_career": True,
        "details_relations": ["currentTeam"]
    })
    
    print("✅ Successfully called.")
    pprint(result)

# Make sure the 'player_id' variable exists from running Cell 1
await test_player_tool(player_id)



Skipping Player Tool test: No Player ID found from the previous cell.


> 3. wyscout_team_info

In [24]:
import asyncio
from pprint import pprint
from datetime import datetime, timedelta

try:
    from backend.agents.wyscout.tools import wyscout_team_info
except ImportError as e:
    print(f"🚨 ImportError: {e}")

def print_header(title):
    print("\n" + "="*80)
    print(f"  🧪 TESTING: {title}")
    print("="*80)

async def test_team_tool(t_id):
    if not t_id:
        print("\nSkipping Team Tool test: No Team ID found from Cell 1.")
        return

    print_header(f"wyscout_team_info (ID: {t_id})")
    
    result = await wyscout_team_info.arun({
        "wyId": t_id,
        "get_squad": True,
        "get_fixtures": True,
        "from_date": (datetime.now() - timedelta(days=90)).strftime('%Y-%m-%d'),
        "to_date": (datetime.now() + timedelta(days=90)).strftime('%Y-%m-%d'),
    })
    
    print("✅ Successfully called.")
    pprint(result)

# Make sure the 'team_id' variable exists from running Cell 1
await test_team_tool(team_id)



Skipping Team Tool test: No Team ID found from Cell 1.


> 4. wyscout_area_list

In [ ]:
import asyncio
from pprint import pprint

try:
    from backend.agents.wyscout.tools import wyscout_area_list
except ImportError as e:
    print(f"🚨 ImportError: {e}")

def print_header(title):
    print("\n" + "="*80)
    print(f"  🧪 TESTING: {title}")
    print("="*80)

async def test_area_list_tool():
    print_header("wyscout_area_list")
    
    result = await wyscout_area_list.arun({})
    
    print("✅ Successfully called. Result snippet:")
    # Print a small part of the large response for readability
    pprint({
        "live_api_areas_snippet": result.get("live_api_areas", [])[:2],
        "documented_custom_areas_snippet": result.get("documented_custom_areas", [])[:2]
    })

await test_area_list_tool()


> 5. wyscout_competition_info

In [ ]:
import asyncio
from pprint import pprint

try:
    from backend.agents.wyscout.tools import wyscout_competition_info
except ImportError as e:
    print(f"🚨 ImportError: {e}")

def print_header(title):
    print("\n" + "="*80)
    print(f"  🧪 TESTING: {title}")
    print("="*80)

season_id = None  # Initialize season_id

async def test_competition_tool(c_id):
    global season_id
    if not c_id:
        print("\nSkipping Competition Tool test: No Competition ID found from Cell 1.")
        return

    print_header(f"wyscout_competition_info (ID: {c_id})")
    
    result = await wyscout_competition_info.arun({
        "wyId": c_id,
        "get_seasons": True,
        "active_seasons_only": True
    })
    
    # Try to extract a seasonId for the next test
    try:
        season_id = result.get('seasons', {}).get('seasons', [{}])[0].get('wyId')
        print(f"✅ Found active Season ID for next test: {season_id}")
    except (IndexError, AttributeError):
        print("⚠️ Could not find an active season ID from the result.")
        
    pprint(result)

# Make sure the 'competition_id' variable exists from running Cell 1
await test_competition_tool(competition_id)


> 6. Season Info

In [ ]:
import asyncio
from pprint import pprint

try:
    from backend.agents.wyscout.tools import wyscout_season_info
except ImportError as e:
    print(f"🚨 ImportError: {e}")

def print_header(title):
    print("\n" + "="*80)
    print(f"  🧪 TESTING: {title}")
    print("="*80)

match_id = None # Initialize variables for later cells
round_id = None

async def test_season_tool(s_id):
    global match_id, round_id
    if not s_id:
        print("\nSkipping Season Tool test: No Season ID found from the previous cell.")
        return

    print_header(f"wyscout_season_info (ID: {s_id})")
    
    result = await wyscout_season_info.arun({
        "wyId": s_id,
        "get_matches": True
    })
    
    # Try to extract a matchId and roundId for later tests
    try:
        first_match = result.get('matches', {}).get('matches', [{}])[0]
        match_id = first_match.get('matchId')
        round_id = first_match.get('roundId')
        print(f"✅ Found Match ID for next test: {match_id}")
        print(f"✅ Found Round ID for next test: {round_id}")
    except (IndexError, AttributeError):
        print("⚠️ Could not find a match ID or round ID from the result.")
        
    pprint(result)

# Make sure the 'season_id' variable exists from running Cell 5
await test_season_tool(season_id)


> 7. wyscout_match_info

In [ ]:
import asyncio
from pprint import pprint

try:
    from backend.agents.wyscout.tools import wyscout_match_info
except ImportError as e:
    print(f"🚨 ImportError: {e}")

def print_header(title):
    print("\n" + "="*80)
    print(f"  🧪 TESTING: {title}")
    print("="*80)

referee_id = None # Initialize for a later cell

async def test_match_tool(m_id):
    global referee_id
    if not m_id:
        print("\nSkipping Match Tool test: No Match ID found from the previous cell.")
        return

    print_header(f"wyscout_match_info (ID: {m_id})")
    
    result = await wyscout_match_info.arun({
        "wyId": m_id,
        "get_details": True,
        "details_relations": ["competition", "teams"]
    })
    
    try:
        referee_id = result.get('details', {}).get('referees', [{}])[0].get('wyId')
        print(f"✅ Found Referee ID for next test: {referee_id}")
    except (IndexError, AttributeError):
        print("⚠️ Could not find a referee ID from the result.")
        
    print("✅ Successfully called.")
    pprint(result)

# Make sure the 'match_id' variable exists from running Cell 6
await test_match_tool(match_id)


> 8. wyscout_round_info

In [ ]:
import asyncio
from pprint import pprint

try:
    from backend.agents.wyscout.tools import wyscout_round_info
except ImportError as e:
    print(f"🚨 ImportError: {e}")

def print_header(title):
    print("\n" + "="*80)
    print(f"  🧪 TESTING: {title}")
    print("="*80)

async def test_round_tool(r_id):
    if not r_id:
        print("\nSkipping Round Tool test: No Round ID found from Cell 6.")
        return

    print_header(f"wyscout_round_info (ID: {r_id})")
    
    result = await wyscout_round_info.arun({
        "wyId": r_id,
        "detail_relations": ["competition", "season"]
    })
    
    print("✅ Successfully called.")
    pprint(result)

# Make sure the 'round_id' variable exists from running Cell 6
await test_round_tool(round_id)


> 9. wyscout_match_events

In [ ]:
import asyncio
from pprint import pprint

try:
    from backend.agents.wyscout.tools import wyscout_match_events
except ImportError as e:
    print(f"🚨 ImportError: {e}")

def print_header(title):
    print("\n" + "="*80)
    print(f"  🧪 TESTING: {title}")
    print("="*80)

async def test_match_events_tool(m_id):
    if not m_id:
        print("\nSkipping Match Events test: No Match ID found from Cell 6.")
        return

    print_header(f"wyscout_match_events (ID: {m_id})")
    
    result = await wyscout_match_events.arun({
        "match_id": m_id,
        "filter_by_primary_types": ["shot", "goal"],
        "filter_by_period": ["1H"]
    })
    
    print("✅ Successfully called (filtered for 1H shots and goals).")
    pprint(result)

# Make sure the 'match_id' variable exists from running Cell 6
await test_match_events_tool(match_id)


> 10. wyscout_advanced_stats

In [ ]:
import asyncio
from pprint import pprint

try:
    from backend.agents.wyscout.tools import wyscout_advanced_stats
except ImportError as e:
    print(f"🚨 ImportError: {e}")

def print_header(title):
    print("\n" + "="*80)
    print(f"  🧪 TESTING: {title}")
    print("="*80)

async def test_advanced_stats_tool(p_id, c_id, s_id):
    if not all([p_id, c_id, s_id]):
        print("\nSkipping Advanced Stats test: Missing player, competition, or season ID.")
        return

    print_header("wyscout_advanced_stats (Player Season Context)")
    
    result = await wyscout_advanced_stats.arun({
        "player_context": {
            "player_id": p_id,
            "competition_id": c_id,
            "season_id": s_id
        }
    })
    
    print("✅ Successfully called.")
    pprint(result)

# Uses IDs from Cells 1 and 5
await test_advanced_stats_tool(player_id, competition_id, season_id)


> 11. wyscout_video_tool

In [ ]:
import asyncio
from pprint import pprint

try:
    from backend.agents.wyscout.tools import wyscout_video_tool
except ImportError as e:
    print(f"🚨 ImportError: {e}")

def print_header(title):
    print("\n" + "="*80)
    print(f"  🧪 TESTING: {title}")
    print("="*80)

async def test_video_tool(m_id):
    if not m_id:
        print("\nSkipping Video Tool test: No Match ID found from Cell 6.")
        return

    print_header(f"wyscout_video_tool (SAFE calls - ID: {m_id})")
    
    result = await wyscout_video_tool.arun({
        "match_id": m_id,
        "check_available_qualities": True,
        "check_period_offsets": True
    })
    
    print("✅ Successfully called for safe info.")
    pprint(result)
    print("\n--- NOTE: Did not test costly 'generate_video_links' action. ---")

# Make sure the 'match_id' variable exists from running Cell 6
await test_video_tool(match_id)


> 12. wyscout_coach_info

In [ ]:
import asyncio
from pprint import pprint

try:
    from backend.agents.wyscout.tools import wyscout_team_info, wyscout_coach_info
except ImportError as e:
    print(f"🚨 ImportError: {e}")

def print_header(title):
    print("\n" + "="*80)
    print(f"  🧪 TESTING: {title}")
    print("="*80)

async def test_coach_tool(t_id):
    if not t_id:
        print("\nSkipping Coach Tool test: No Team ID found from Cell 1.")
        return
        
    print_header(f"wyscout_coach_info (finding coach from team {t_id})")
    
    # First, get the squad to find a coach ID
    squad_result = await wyscout_team_info.arun({"wyId": t_id, "get_squad": True})
    coach_id = None
    try:
        coach_id = squad_result.get('squad', {}).get('coach', [{}])[0].get('wyId')
        print(f"✅ Found Coach ID: {coach_id}")
    except (IndexError, AttributeError):
        print(f"⚠️ Could not find a coach for team {t_id}.")
        return

    # Now test the coach tool with the found ID
    result = await wyscout_coach_info.arun({
        "wyId": coach_id,
        "detail_relations": ["currentTeam"]
    })
    
    print("✅ Successfully called.")
    pprint(result)

# Make sure the 'team_id' variable exists from running Cell 1
await test_coach_tool(team_id)


> 13. wyscout_referee_info

In [ ]:
import asyncio
from pprint import pprint

try:
    from backend.agents.wyscout.tools import wyscout_referee_info
except ImportError as e:
    print(f"🚨 ImportError: {e}")

def print_header(title):
    print("\n" + "="*80)
    print(f"  🧪 TESTING: {title}")
    print("="*80)

async def test_referee_tool(r_id):
    if not r_id:
        print("\nSkipping Referee Tool test: No Referee ID found from Cell 7.")
        return

    print_header(f"wyscout_referee_info (ID: {r_id})")
    
    result = await wyscout_referee_info.arun({
        "wyId": r_id,
        "include_image_data": False # Set to False to keep output clean
    })
    
    print("✅ Successfully called.")
    pprint(result)

# Make sure the 'referee_id' variable exists from running Cell 7
await test_referee_tool(referee_id)
